# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda

Vamos trabalhar com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('previsao_de_renda.csv')

In [ ]:
df.info()

1. Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
    - Utilizando os recursos do Patsy, coloque as variáveis qualitativas como *dummies*.
    - Mantenha sempre a categoria mais frequente como casela de referência
    - Avalie os parâmetros e veja se parecem fazer sentido prático.  


2. Remova a variável menos significante e analise:
    - Observe os indicadores que vimos, e avalie se o modelo melhorou ou piorou na sua opinião.
    - Observe os parâmetros e veja se algum se alterou muito.  


3. Siga removendo as variáveis menos significantes, sempre que o *p-value* for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor. 
    

##### Ajuste um modelo para prever log(renda) considerando todas as covariáveis disponíveis.
##### Utilizando os recursos do Patsy, coloque as variáveis qualitativas como dummies

In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

# Carregar a base de dados
renda = pd.read_csv('previsao_de_renda.csv')

# Verificar nomes das colunas disponíveis
print("Colunas disponíveis:", renda.columns.tolist())

# Remover colunas apenas se existirem
for col in ['data_ref', 'index']:
    if col in renda.columns:
        renda = renda.drop(columns=col)

# Criar as matrizes de design (X e y)
y, X = dmatrices(
    'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia',
    data=renda, return_type='dataframe'
)

# Ajustar o modelo de regressão linear múltipla
modelo_completo = sm.OLS(y, X).fit()

# Exibir o resumo do modelo
print(modelo_completo.summary())

Colunas disponíveis: ['Unnamed: 0', 'data_ref', 'id_cliente', 'sexo', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos', 'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia', 'idade', 'tempo_emprego', 'qt_pessoas_residencia', 'renda']
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Thu, 30 Oct 2025   Prob (F-statistic):               0.00
Time:                        14:37:14   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust        

O for col in ['data_ref', 'index'] garante que a função só tenta excluir colunas que realmente existem no DataFrame.
O print(renda.columns.tolist()) ajuda a conferir os nomes reais das coluna.
O resto do código continua igual, criando as variáveis dummies automaticamente via Patsy e ajustando o modelo de regressão.

##### Mantenha sempre a categoria mais frequente como casela de referência

In [5]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

# Carregar a base
renda = pd.read_csv('previsao_de_renda.csv')

# Encontrar a categoria mais frequente em cada variável categórica
categoricas = ['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'tipo_renda',
               'educacao', 'estado_civil', 'tipo_residencia']

for var in categoricas:
    mais_freq = renda[var].mode()[0]  # categoria mais frequente
    renda[var] = pd.Categorical(renda[var],
                                categories=[mais_freq] + [c for c in renda[var].unique() if c != mais_freq],
                                ordered=True)

# Criar as matrizes de design com Patsy — agora com a categoria mais frequente como referência
y, X = dmatrices(
    'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia',
    data=renda, return_type='dataframe'
)

# Ajustar o modelo
modelo = sm.OLS(y, X).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     287.5
Date:                Thu, 30 Oct 2025   Prob (F-statistic):               0.00
Time:                        14:50:50   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.737e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercep

##### Avalie os parâmetros e veja se parecem fazer sentido prático.
##### Remova a variável menos significante e analise:

In [7]:
import statsmodels.api as sm
from patsy import dmatrices

# Fórmula inicial com todas as variáveis
formula_base = 'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia'

def backward_elimination(formula, data, p_limite=0.05):
    y, X = dmatrices(formula, data=data, return_type='dataframe')
    modelo = sm.OLS(y, X).fit()
    
    while True:
        # P-valores do modelo
        p_valores = modelo.pvalues.drop("Intercept", errors='ignore')
        p_max = p_valores.max()
        if p_max > p_limite:
            variavel_remover = p_valores.idxmax()
            
            # Extrair apenas o nome da variável base antes de "[T."
            base_var = variavel_remover.split("[")[0]
            
            # Remover todas as dummies associadas à variável base
            termos = formula.split("~")[1]
            termos_lista = [v.strip() for v in termos.split("+") if base_var not in v]
            nova_formula = "np.log(renda) ~ " + " + ".join(termos_lista)
            
            # Reajustar modelo
            y, X = dmatrices(nova_formula, data=data, return_type='dataframe')
            modelo = sm.OLS(y, X).fit()
            formula = nova_formula
        else:
            break
    
    return modelo, formula

# Executar o processo
modelo_final, formula_final = backward_elimination(formula_base, renda)

print("📉 Fórmula final:", formula_final)
print(modelo_final.summary())

📉 Fórmula final: np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + idade + tempo_emprego
                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     1098.
Date:                Thu, 30 Oct 2025   Prob (F-statistic):               0.00
Time:                        14:58:16   Log-Likelihood:                -13673.
No. Observations:               12427   AIC:                         2.736e+04
Df Residuals:                   12420   BIC:                         2.741e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
-----

Após o ajuste dos modelos, o modelo reduzido apresentou melhor equilíbrio entre simplicidade e desempenho estatístico. Embora o R² ajustado tenha se mantido muito próximo ao do modelo completo (0,347 contra 0,356), o modelo final apresentou menores valores de AIC e BIC, indicando melhor qualidade de ajuste com menor complexidade. Além disso, todas as variáveis remanescentes — sexo, posse de veículo, posse de imóvel, número de filhos, idade e tempo de emprego — mostraram-se estatisticamente significativas e coerentes do ponto de vista econômico, representando fatores plausíveis de influência sobre a renda. Assim, o modelo reduzido é considerado o mais adequado por ser mais parcimonioso, interpretável e robusto

##### Observe os parâmetros e veja se algum se alterou muito.

Ao comparar os parâmetros do modelo completo e do modelo reduzido, verifica-se que as magnitudes e os sinais dos coeficientes principais permaneceram consistentes, indicando que a exclusão das variáveis não significantes não alterou substancialmente as relações entre as variáveis explicativas e a renda. As pequenas variações observadas são esperadas e refletem a redução da multicolinearidade, resultando em estimativas mais estáveis e precisas. Conclui-se, portanto, que a simplificação do modelo preservou a coerência e a robustez dos resultados.

##### Siga removendo as variáveis menos significantes, sempre que o p-value for menor que 5%. Compare o modelo final com o inicial. Observe os indicadores e conclua se o modelo parece melhor.

Após a remoção iterativa das variáveis menos significantes (p > 0,05), o modelo final apresentou um R² ajustado ligeiramente menor (0,347 contra 0,356 no modelo inicial), porém valores menores de AIC e BIC, o que indica melhor equilíbrio entre ajuste e parcimônia. Além disso, todos os coeficientes restantes mostraram-se estatisticamente significativos e coerentes do ponto de vista econômico.

Conclui-se que o modelo final é mais simples, estável e eficiente, mantendo poder explicativo semelhante ao modelo completo, mas com menor complexidade e maior robustez estatística